# <span id="chap_generating_functions"></span> Generating arbitrary topologies: generating functions

Probability mass functions, the configuration model, and indeed many other ways of specifying a probability distribution, are structured around the treatment of individual values in that distribution. While this is an essential part of a distribution, it is not the only factor of interest. Typically we are interested in a distribution's summary statistics &ndash; its mean and variance &ndash; and perhaps in other structures within it. If the distribution represents the degrees of nodes in a network, we are interested not simply in the nodes themselves, but in what other structures arise as a result. In particular we may be interested in the formation of connected components of nodes, or in questions like: if we pick a random node, what are the likely degrees of its neighbours? While these properties are immanent in the probability distribution, it is not immediately clear how they relate to it, especially in the case of networks with arbitrary degree distribution.

Fortunately there is a way of representing a distribution that makes higher-order structures far easier to extract. [Generating functions](https://en.wikipedia.org/wiki/Generating_function) are a mathematical device for encapsulating a collection of individual discrete probabilities into a single compound function. Wilf [[Wil90]](bibliography.ipynb#Wil90) describes them as "a clothes line on which to hang a series": they are somewhat related to [Taylor series](https://en.wikipedia.org/wiki/Taylor_series), which (I was delighted to discover in the course of writing this) themselves originate in the work of another St Andrews academic, [James Gregory](https://en.wikipedia.org/wiki/James_Gregory_%28astronomer_and_mathematician%29).

The standard treatment of generating functions [[NSW01]](bibliography.ipynb#NSW01) is complete, but rather abstract. Following our usual approach, we'll take both the mathematical *and* the computer science view, which will hopefully make the key ideas clearer to a wider audience. While they feel superfluous and unecessarily arcane at the start, it *is* worth hanging in to understand the full power of generating functions: they allow some very complex ideas to be expressed and computed with very easily. Moreover they are used extensively in the literature, so understanding them makes a lot of research much more accessible. 

### Definition and construction

In discussing probability mass functions above, we represented the distribution as a sequence of values $p_k$ where each $p_k$ is the probability of the integer value $k$ appearing in the distribution. To represent this as a [probability generating function](https://en.wikipedia.org/wiki/Probability-generating_function), we take the series $p_k$ construct a function $G_0$ of the form: 

$$
    G_0(x) = \sum_k p_k x^k
$$

In this function, $x$ is simply a parameter that runs from $0$ to $1$. The distribution has to be properly normalised, so $\sum_k p_k = 1$. In some circumstances we may have measurements for the number of nodes $N_k$ for each degree $k$. In this case we can rephrase the definition to give:

$$
    G_0(x) = \frac{\sum_k N_k x^k}{\sum_k N_k}  = \frac{\sum_k N_k x^k}{N}
   $$

where the denominator normalises the distribution by dividing by the number of nodes in the network.

To be clear, the generating function $G_0(x)$ of a random variable $X$ is a series of powers, where the coefficient of the term in $x^k$ is the probability that $X = k$. If we know these values, we can easily chain them together to construct the generating function. The sequence may be finite, if there are only a finite number of values of $k$ that $X$ can take; or it may be infinite. The sequence may be enumerated explicitly, or may be created from some other function of $k$ as we'll see later.

### Evaluating the generating function

This seems like a strange sort of equation that leads to all kinds of questions: where do the powers of $x$ come from, and what does $x$ &ndash; and indeed $G_0$ itself &ndash; represent at all? The answers to these questions are not at all obvious.

Firstly, let is deal with $x$ by looking at the extremal values. If we evaluate $G_0$ at $x = 0$ we find:

$$
\begin{align*}
    G_0(0) &= \sum_k p_k x^k \\
           &= \sum_k p_k 0^k \\
           &= p_0 \times 1 + p_1 \times 0 + p_2 \times 0 \cdots \\
           &= p_0
\end{align*}
$$

So $G_0(0) = p_0$, the probability that $X = 0$ or, in our case, that a node chosen at random from the network is isolated. At the other extreme,

$$
\begin{align*}
    G_0(1) &= \sum_k p_k 1^k \\
           &= \sum_k p_k \\
           &= 1
\end{align*}
$$

$G_0(1)$ is $1$ by definition, since it is the sum of all the component probabilities.

What about intermediate values of $x$? *We'll never use them*.

A generating function is what's termed a *formal power series*. It combines information together using powers of a parameter, but is never intended to be generally evaluated. Instead, it's intended as an encoding to be *manipulated*, and the encoding is such that it allows some simple operations to be performed that would otherwise be complicated.

### <span id="sec_generating_functions_encoding"></span> The encoding

So from a computer science perspective, what kind of encoding is the generating function, and what are the operations we can perform on it?

The generating function is composed of a sequence of component probabilities, each multiplied by a power of $x$. Suppose we have a finite sequence of such probabilities, *i.e.*, there are only certain values of $k$ possible. Since the powers of $x$ are directly related to the position of a term in the sequence, we might think of this sequence as a list where the $k$th element corresponds to $p_k$:

In [19]:
# a sample generating function
#   p_0 = 0.1
#   p_1 = 0.2
#   p_2 = 0.5
#   p_3 = 0.2
g0 = [ 0.1, 0.2, 0.5, 0.2 ]

We can then look at the operations we do on the generating function and try to understand them by translating them into our list representation.

To evauate the generating function we have to multiply each element by the corresponding power of $x$ and sum the results. The $k$th element of the sequence is multipled by $x^k$:

In [46]:
def gf_evaluate( g, x ):
    """Evaluate the generating function.
    
    g: the generating function as a sequence
    x: the parameter
    returns: the value"""
    v = 0
    xx = 1
    for k in range(len(g)):
        v = v + g[k] * xx
        xx = xx * x
    return v

Following Wilf's comment from above, the list is the "clothes line" onto which we've "hung" the component probabilities. We have already seen two operations, namely evaluating $G_0(x)$ at the extremal values of 0 and 1. Seen as a sequence operation, $G_0(0)$ extracts the first element of the sequence of probabilities (the head of the sequence list):

In [47]:
print gf_evaluate(g0, 0)

0.1


At the other extreme, $G_0(1)$ returns the sum of the sequence:

In [27]:
print gf_evaluate(g0, 1)

1.0


And we never evaluate the generating function at other values of $x$.

How can we access other values of $p_k$? One other operation we can perform is differentiation. (We use the "tick" notation $G_0'(x)$ to denote $\frac{d}{dx} G_0(x)$.)

\begin{align*}
    G_0'(x) = \frac{d}{dx} G_0(x) &= \frac{d}{dx} \sum_k p_k x^k \\
                                  &= \frac{d}{dx} p_0 + p_1 x + p_2 x^2 + p_3 x^3 + \cdots \\
                                  &= p_1 + 2 p_2 x + 3 p_3 x^2 + \cdots
\end{align*}

So differentiation multiplies each element of the sequence by its place in the sequence and then moves the new sequence one place to the left, discarding the first element:

In [48]:
def gf_differentiate( g ):
    """Differentiate the generating function.
    
    g: the generating function
    returns: the derived function"""
    dg = g[:]
    for k in range(len(g)):
        dg[k] = g[k] * k
    return dg[1:]

print gf_differentiate(g0)

[0.2, 1.0, 0.6000000000000001]


Remember, the power of $x$ in the generating function corresponds to the position of the coefficient in the sequence.

A final operation is to multiply through by $x$, which moves each element one place to the right and adds 0 at the front:

In [29]:
def gf_multiplyx( g ):
    """Multiply the generating function through by x.
    
    g: the generating function
    returns: the new function"""
    return [ 0 ] + g

print gf_multiplyx(g0)

[0, 0.1, 0.2, 0.5, 0.2]


So the generating function provides an encoding of the probability distribution that corresponds to a sequence of component probability masses, and the four operations we can perform on it &ndash; evaluate at 0, evaluate at 1, differentiate, and multiply through &ndash; form an abstract data type that is similar (but not identical) to the usual list type.

### Retrieving the component probabilities

Now we understand the generating function encoding better, we can look more closely at some of the operations we might want to perform on the distribution it represents. Chief among these is retreving the component probabilities: for a given $k$, what is $p_k$? Phrased in terms of the sequence encoding, we can only directly extract $p_0$. But we can also move the sequence left by differentiation, and this leads to an obvious algorithm: to extract the value of $p_k$, move the sequence left $k$ places and then evaluate the resulting generating function at 0.

It's not quite that simple, though, as differentiation doesn't just move the sequence left: it also multiplies each coefficient by its place in the sequence as it does so. If we take derivatives $k$ times, the head of the sequence (the value at $x^0$) will be $p_k$ multiplied $k(k - 1)(k - 2) \ldots 1 = k!$. So to extract $p_k$ we perform repeated differentiation, evaluate the resulting function at 0, and then divide by $k!$:

$$
    p_k = \frac{1}{k!} \bigg ( \frac{d}{dx} \bigg )^k G_0(x) \, \bigg |_{x = 0}
$$

Just to check, let's extract $p_2$ from our generating function above:

\begin{align*}
    p_2 &= \frac{1}{k!} G_0''(0) \\
        &= \frac{1}{k!} \frac{d^2}{dx^2} (0.1 + 0.2 \times x + 0.5 \times x^2 + 0.2 \times x^3) \, \bigg |_{x = 0} \\
        &= \frac{1}{k!} \frac{d}{dx} (0.2 + 1 \times x + 0.6 \times x^2) \, \bigg |_{x = 0} \\
        &= \frac{1}{k!} (1 + 1.2 \times x) \, \bigg |_{x = 0} \\
        &= \frac{1}{2!} \\
        &= 0.5
\end{align*}

And computationally:

In [35]:
print gf_evaluate(gf_differentiate(gf_differentiate(g0)), 0) / math.factorial(2)

0.5


If this seems like a convoluted way to access the $k$th element of a list, involving various spuriousy complicated mathematical operations, then &ndash; well, it is. That's not quite the point, however. The generating function does make other operations considerably more straightforward and mathematically tractable while remaining computationally very cheap. Let's look at some of these other operations in more detail.  

### Summary statistics

The generating funciton represents the distribution as a whole, and can be used directly to extract information about the distribution as a whole: the summary statistics like mean and variance, also called the moments of the distribution.

The mean degree $\langle k \rangle$ corresponds to the first moment and is defined as $\langle k \rangle = \sum_k k \, p_k$: in other words, multiplying each component probability by the degree it corresponds to and summing the results. Looking back at the [encoding](#sec_generating_functions_encoding), we have operations available to perform each of these steps: differentiation to do the multiplications, and evaluating at 1 to perform the summation. So we can determine the mean degree directly from the generating function:

$$
    \langle k \rangle = G_0'(1)
$$

Does it matter that we lost $p_0$ in the differentiation? No, for two reasons. Firstly, the definition of the mean would perform $p_0 \times 0 = 0$ anyway; secondly, the remaining values of $p_k$ sum to $1 - p_0$, so all the information is retained.

The second moment of the distribution is its variance, defined by $\langle k^2 \rangle = \sum_k k^2 p_k$. Clearly this involves roughly the same operation, but there's a problem: each differentiation moves the sequence to the left, so differentiating twice doesn't multiply $p_k$ by $k$ twice, it multiplies $p_k$ by $k$ and then by $(k - 1)$. We used this property when extracting the $p_k$ from the generating function, but now it's in the way. We can, however, differentiate to perform the multiplication and then move the sequene right again by multiplying through by $x$ &ndash; essentially multiplying each $p_k$ by $k$ while leaving it in the same position alongside the appropriate power of $x$. Putting this together:

$$
    \langle k^2 \rangle = \bigg ( x \frac{d}{dx} \bigg ) \bigg ( x \frac{d}{dx} \bigg ) G_0(x) \, \bigg |_{x = 1}
$$

and in general for the $n$th moment:
 
$$
    \langle k^n \rangle = \sum_k k^n \, p_k = \bigg ( x \frac{d}{dx} \bigg )^n G_0(x) \, \bigg |_{x = 1}
$$

In computational form, we have:

In [49]:
# mean from component probabilities
m = 0
for k in range(len(g0)):
    m = m + k * g0[k]  # k * p_k
print "mean degree of g0 (by hand) = {m}".format(m = m)

# mean and variance from generating function
print "mean degree of g0 = {m}".format(m = gf_evaluate(gf_multiplyx(gf_differentiate(g0)), 1))
print "variance of g0 = {var}".format(var = gf_evaluate(gf_multiplyx(gf_differentiate(gf_multiplyx(gf_differentiate(g0)))), 1))

mean degree of g0 (by hand) = 1.8
mean degree of g0 = 1.8
variance of g0 = 4.0


This still seems rather complicated compared to the purely computational rendering, but we're not finished with the properties of generating functions yet.

### <span id="sec_generating_functions_powers"></span> Powers

The real power of generating functions starts to become apparent when we look at more complex statistical operations. Suppose, for example that we want to choose $m$ values from a distribution. What is the expected distribution of these values? Clearly they derive from the distribution of single values, but how?

Generating functions make this calculation easy. Without going into the details &ndash; which are complex but not interesting or illuminating of anything important &ndash; we can observe that if we have a property $a$ derived from the distribution described by $G_0(x)$, then $m$ independent realisations of that property will be given by the distribution described by $[ G_0(x) ]^m$, the $m$th power of $G_0(x)$.

[MORE]

### The generating functions for ER and power law networks

We said that the start that the coefficients on the "clothes line" of the generating function could be enumerate, or could be computed by some other function of $K$. So far we'ver seen only the first form, but we've encountered networks for which the second form woul be appropriate. The [ER network](er_networks.ipynb), for example, has a degree distribution given by:

$$
    p_k = \frac{\langle k \rangle^k e^{-\langle k \rangle}}{k!}
$$

in a large network. In other words, there is a function of $k$ to determine the coefficient of $x^k$ in the generating function representation. We can therefore construct the generating function by substituting this equation into the general equation above, to get:

\begin{align*}
    G_0(x) &= \sum_{k = 0}^\infty \frac{\langle k \rangle^k e^{-\langle k \rangle}}{k!} x^k \\
           &= e^{-\langle k \rangle} \sum_{k = 0}^\infty \frac{\langle k \rangle^k x^k }{k!} \\
\end{align*}

(Remember that $\langle k \rangle$ is the mean degree and so is a constant.) This looks fearsome, but if we compare it to the series definition of the exponential function, we have:

\begin{align*}
    e^{cx} &= 1 + cx + \frac{c^2 x^2}{2!} + \frac{c^3 x^3}{3!} + \cdots \\
           &= \sum_{k = 0}^\infty \frac{c^k x^k}{k!}
\end{align*}

and therefore:

\begin{align*}
    G_0(x) &= e^{-\langle k \rangle} \sum_{k = 0}^\infty \frac{\langle k \rangle^k x^k}{k!} \\
           &= e^{-\langle k \rangle} \, e^{\langle k \rangle x} \\
           &= e^{\langle k \rangle (x - 1)}
\end{align*}

Let's pause a moment to see what all this maths has bought us. We've taken an expression for node degrees and strung it together as a power series to get a generating function. We've then noticed that the resulting expression bears a resemblance to *another* power series, the exponential function, have re-arranged the former to match the latter, and therefore managed to express the generating function in terms of exponentials &ndash; and in the process, we've hidden the power series aspect entirely. Does this matter? No, the point of the generating function representation is that we can apply the *same* operations to a generating function specified in *any* way internally, and it'll still work. For example, extracting the mean degree from the generating function involves evaluating $G_0'(1)$. We therefore have:

$$
\begin{align*}
    \frac{d}{dx} G_0(x) \, \bigg |_{x = 1}
    &= \frac{d}{dx} e^{\langle k \rangle (x - 1)} \bigg|_{x = 1} \\
    &= \bigg ( \frac{d}{dx} \langle k \rangle (x - 1)  \bigg ) e^{\langle k \rangle(x - 1)} \, \bigg |_{x = 1} \\
    &= \langle k \rangle e^{\langle k \rangle(x - 1)} \, \bigg |_{x = 1} \\
    &= \langle k \rangle e^0 \\
    &= \langle k \rangle
\end{align*}
$$

(where the second step is the standard way of differentiating an exponential).

What about for a [power law network](powerlaw.ipynb)? In this case we have that $p_k = C \, k^{-\alpha}$ where $C = \frac{1}{\zeta(\alpha, 1)}$, the zeta "special" function, and the corresponding generating function is therefore:

$$
G_0(x) = C \sum_{k} \frac{x^k}{k^\alpha}
$$

This sum is an example of another "special" function, a [polylogarithm](https://en.wikipedia.org/wiki/Polylogarithm) of order $\alpha$, denoted $Li_\alpha(x)$ and computable directly in Python using `Scipy`'s `mpmath.polylog(alpha, x)` function.

### Neighbouring nodes and excess degree

So generating functions provide a uniform representation for degree distributions defined piecewise and using a formula. But their power really comes to the fore when dealing with derived distributions, to which we will now turn our attention.

Suppose we choose a node $v$ from a graph, and then follow a randomly-chosen edge $e$ to another node $w$: what would we expect the degree of $w$ to be? One might na&iuml;vely expect the degree of $w$ to be the mean degree of the network, $\langle k \rangle$. However, this isn't quite true: the network may contain isolated nodes (of degree 0), which we can't get to by traversing $e$. Moreover, there are more ways to get to a node with high degree than to get to a node of low degree, so we might expect that the distribution of the degrees of neighbouring nodes will be skewed somewhat: a randomly-selected node is more likely to be adjacent to a high-degree node than a low-degree one, simply because high-degree nodes have more neighbours.

To make this formal, let's return to using the configuration model. When constructing a network in this model, we start with each node of degree $k$ having $k$ "stubs" that are then connected together. There are therefore $\langle k \rangle N$ stubs in the network, and each edge connects two stubs.

Suppose we choose two nodes $i$ and $j$ at random. What is the probability of them being neighbours? For each stub on $i$ there are $\langle k \rangle N - 1$ other stubs it can be connected to. If we denote the degree of $i$ and $j$ by $k_i$ and $k_j$ respectively, then of these $\langle k \rangle N - 1$ stubs, $k_j$ of them are on node $j$, so the probability of $i$ being connected to $j$ by $e$ is simply $\frac{k_j}{\langle k \rangle N - 1}$. Since there are $k_i$ edges by which $i$ could be connected to $j$, the probability of $i$ and $j$ being adjacent is given by $k_i \frac{k_j}{\langle k \rangle N - 1} \approx \frac{k_i k_j}{\langle k \rangle N}$. 

This covers the case of a *specific* node with degree $k_j$: what about *any* such node? In a simple network without self loops, a node $j$ of degree $k_j$ is adjacent to $k_j$ other nodes. There are $N - 1$ nodes it can be adjacent to, so the probability of one of those nodes being our starting node $i$ is $\frac{k_j}{\langle k \rangle N - 1}$. There are by definition $N p_{k_j}$ nodes of degree $k_j$ in the network, so the probability of $i$ being adjacent to *some* such node is given by $\frac{k_j}{\langle k \rangle N - 1} N p_{k_j} \approx \frac{k_j p_{k_j}}{\langle k \rangle}$. Let's lose some of the subscripts and say that the probability of a randomly-chosen node being adjacent to *some* node of degree $k$ is given by $\frac{k p_k}{\langle k \rangle}$.

(Notice what we did in that last paragraph: we switched from talking about the degree of our randomly-chosen node to talking instead about the degree of the nodes to which it might be adjacent. It appears, somewhat counter-intuitively, that the probability of a randomly-chosen node being adjacent to a node of degree $k$ isn't affected by the degree of the randomly-chosen node itself. It's better to think of this as being taken care of by the $\langle k \rangle$ factor: we expect the randomly-chosen node to have degree $\langle k \rangle$, and *given this* it is more likely that the node is adjacent to nodes that are, by definition, adjacent to more nodes.)

Having got this far, we can now construct a generating function that describes the probability distribution of the nodes that are neighbours of a randomly-chosen node. We have several elements to each component probability. It is multiplied by its degree; however, we want the probability to stay associated with the correct degree; and finally, we need to divide each probability by the mean degree of the distribution. Looking back to the sequence encoding, this suggests a differentiation, a multiplication by $x$, and a division by $G_0'(1)$. For a given node, the distribution of degrees of a neighbour is given by

$$
    x \frac{G_0'(x)}{G_0'(1)}
$$

Often, however, we don't actually want the degree of neighbours. Instead we want the degree of neighbours *excluding ourselves*, since we know we're a neighbour already. Put another way, we'd like a probability distribution for the number of neighbours of neighbours we might have. This is referred to as the <span id="def_excess_degree">*excess degree*</span> of a neighbouring node. We can get this easily. If a node has degree 1, and is our neighbour, then its excess degree must be 0 (since it can have no other neighbours); similarly, a node of degree 2 has one other neighbour (excluding ourselves); and so forth. Therefore we want a probability $p_k$ that a node has degree $k$ to be associated with an excess degree of $k - 1$. In terms of generating functions, we want to shift the distribution one to the left, which we can do by letting the differential operator work unimpeded. We thus end up with a generating function for the excess degree of the node neighbouring a randomly-chosen node being given by:

$$
    G_1(x) = \frac{G_0'(x)}{G_0'(1)} = \frac{1}{\langle k \rangle} G_0'(x)
$$

There are two points to make about this. Firstly, it is a purely generating-function result and so applies to *any* distribution for which we have a generating function. Secondly, the "powers" property of generating functions means that we can iterate to find the distribution of second neighbours:

\begin{align*}
    G_2(x) &= \sum_k p^k [ G_1(x) ]^k \\
           &= G_0(G_1(x))
\end{align*}

and third neighbours:

$$
    G_3(x) = G_0(G_1(G_1(x)))
$$

Since these are just generating functions themselves, we can still use the same identities we used above, so that the average number of second neighbours is given by:

\begin{align*}
    \langle k_2 \rangle &= \frac{d}{dx} G_2(x) \, \bigg |_{x = 1} \\
                        &= G_0'(1) G_1'(1) \\
                        &= G_0''(1)
\end{align*}

or, more computationally,

In [67]:
print gf_evaluate(gf_differentiate(gf_differentiate(g0)), 1)

2.2


Again, the important point is that these techniques work for *any* generating function. Once we have the base distribution, we can derive important other distributions mechanically. In this section we have worked with excess degree, but the same approach applies to finding component sizes.

### Components

A network will typically have zero or more *components*, in which all nodes are reachable from all others by following some path of edges through the cluster. We can use the generating functions of the degree distribution, and the other structures derived from them, to work out the distribution of cluster sizes in networks with different degree structures.

We can phrase this in a similar way to the way we phrased the degree distribution of neighbours above: if we follow an edge at random to a node $w$, what is the probability distribution of the cluster to which $w$ belongs? We'll refer to this function as $H_1(x)$, by analogy to the earlier functions: $G_0(x)$ is the generating function for edges at a node, $G_1(x)$ the distribution of the edges of nodes that are neighbours of a randomly-chosen node, $G_2(x)$ the distribution of edges on second-neighbours, and so on. $H_1(x)$ is the distribution of the clusters to which neighbouring nodes may belong.

Think about what happens as we follow $e$. We end up at a node $w$ that has degree at least $1$ (since we traversed $e$ to reach it. It may have no neighbours, or it may have neighbours which themselves have neighbours and so on. In the absence of loops we can imagine these neighbours as forming a tree whose branches are all the neighbouring nodes of $w$ except the one attached to $e$ down which we arrived.

In terms of probabilities, let $q_k$ denote the probability that $w$ has $k$ neighbours *excluding the one we came from*. This probability is generated by $G_1(x)$, since it is the probability of the degree of a neighbour of a randomly-chosen node.

### Shortest paths

Another useful property of a graph is the average shortest path between two randomly-chosen nodes.

We can consider this problem in terms of the number of nodes encountered as we follow a path. From the starting node we traverse a random edge (one of $\langle k \rangle$) and encounter another node, which in turn has $\langle k^2 \rangle$ neighbours, and so on. If we continue this process, keeping track of how many neighbouring nodes we encounter as we go (and ignoring the possibility of loops as mentioned above), then eventually the total number of nodes encountered will equal the number of nodes in the graph, and there will therefore have been a path between the initial node and any other node of about half this length. The shortest path $l$ is therefore found by solving:

$$1 + \sum_{m = 1}^{l} \langle k^m \rangle = N$$

Using the large-graph approximation we can reduce this to:

$$l \approx 1 + \frac{log(\frac{N}{\langle k \rangle})}{log (\frac{\langle k^2 \rangle}{\langle k \rangle})} \approx \frac{log N}{log \langle k^2 \rangle}$$


## Summary

 * $G_0(x)$ &ndash; generating function for node degrees
 * $G_0^{(k)}(0)$ &ndash; probability of a node of degree $k$
 * $G_0'(1)$ &ndash; average degree distribution $\langle k \rangle$
 * $G_1(x)$ &ndash; generating function for degrees of first neighbours of a randomly-chosen node